In [ ]:
"""This script evaluates an LLM prompt for processing text so that it can be used for the wttr.in API"""

from ollama import Client

LLM_MODEL: str = "gemma3:27b"
client: Client = Client(
  host='http://10.1.69.214:11434/'
)

# TODO: define  llm_parse_for_wttr()
def llm_parse_for_wttr():
    

# Test cases
test_cases = [ # TODO: Replace these test cases with ones for wttr.in
    {
        "input": "Extract the number: The total is 42 units.",
        "expected": "42"
    },
    {
        "input": "What color is the fruit: banana.'",
        "expected": "Yellow"
    },
    {
        "input": "Give me the JSON: Name is Alice, age is 30",
        "expected": '{"name": "Alice", "age": 30}'
    },
]

IndentationError: expected an indented block after function definition on line 11 (1302159875.py, line 14)

In [ ]:
# Function to iterate through test cases
def run_tests():
    num_passed = 0

    for i, test in enumerate(test_cases, 1):
        raw_input = test["input"]
        expected_output = test["expected"]

        print(f"\nTest {i}: {raw_input}")
        try:
            result = llm_parse_for_wttr(raw_input).strip()
            expected = expected_output.strip()

            print("LLM Output  :", result)
            print("Expected    :", expected)

            if result == expected:
                print("✅ PASS")
                num_passed += 1
            else:
                print("❌ FAIL")

        except Exception as e:
            print("💥 ERROR:", e)

    print(f"\nSummary: {num_passed} / {len(test)} tests passed.")

In [ ]:
# Run the test cases
run_tests()